In [2]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
import pandas as pd
import numpy as np
import gc
import time
from pathlib import Path
from tqdm.auto import tqdm
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    matthews_corrcoef, brier_score_loss
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from tensorflow.keras import models, layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping

import shap
import optuna

# ============================================================
# ⚙️ Global Configuration
# ============================================================
base_dir = Path().resolve()
results_dir = base_dir / "results"
results_dir.mkdir(exist_ok=True)

TUNE_MODE = True   # ✅ set False for quick run
N_TRIALS = 20
SEED = 42

print(f"🔧 TUNE_MODE: {'ON' if TUNE_MODE else 'OFF'} | Optuna Trials: {N_TRIALS}")

🔧 TUNE_MODE: ON | Optuna Trials: 20


In [11]:
# Select Number of Feature for Each Modality
tr_n = 250
fl_n = 200

In [5]:
# Evaluation Function for each model
def evaluate_model(model, X_test, y_test, model_name):
    """Evaluate multiple classification metrics."""
    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_test)[:, 1]
        y_pred = (y_prob > 0.5).astype(int)
    else:
        y_prob = model.predict(X_test)
        y_pred = (y_prob > 0.5).astype(int)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)
    mcc = matthews_corrcoef(y_test, y_pred)
    brier = brier_score_loss(y_test, y_prob)

    print(f"✅ {model_name}: acc={acc:.3f}, f1={f1:.3f}, auc={auc:.3f}, mcc={mcc:.3f}, brier={brier:.3f}")
    return acc, prec, rec, f1, auc, mcc, brier

In [9]:
# Load Transcriptomics and Fluxomics data
def load_transcriptomic(base_dir, meta):
    trans = pd.read_csv(base_dir.parent / "dataset/transcriptomic_hvg.csv", index_col=0)
    trans = pd.merge(trans, meta[["response"]], left_index=True, right_index=True, how="inner")
    trans = trans.loc[:, ~trans.columns.str.upper().str.startswith("MT-")]
    trans["response"] = trans["response"].map({"Responder": 1, "Non-responder": 0})
    y = trans["response"].astype(np.int8)
    X = trans.drop(columns=["response"]).astype(np.float32)
    return X, y


def load_fluxomic(base_dir):
    flux = pd.read_csv(base_dir.parent / "dataset/fluxomics.csv", index_col=0)
    rxns = [c for c in flux.columns if c != "response" and not c.startswith("EX_")]
    flux = flux[rxns + ["response"]]
    flux["response"] = flux["response"].map({"Responder": 1, "Non-responder": 0})
    y = flux["response"].astype(np.int8)
    X = flux.drop(columns=["response"]).astype(np.float32)
    return X, y

In [12]:
# Load Meta Data and select features
meta = pd.read_csv(base_dir.parent / "dataset/metadata.csv", index_col=0)
hvg_genes = pd.read_csv(base_dir.parent / "dataset/hvg_genes.csv", header=None).squeeze().tolist()

X_trans, y_trans = load_transcriptomic(base_dir, meta)
X_flux, y_flux = load_fluxomic(base_dir)

# select top features
tr_top = [g for g in hvg_genes if g in X_trans.columns][:tr_n]
fl_top = X_flux.var().sort_values(ascending=False).index[:fl_n]

X_tr_250 = X_trans[tr_top]
X_fl_200 = X_flux[fl_top]

# multimodal merge
X_multi_full = pd.merge(X_trans, X_flux, left_index=True, right_index=True, how="inner")
y_multi = meta.loc[X_multi_full.index, "response"].map({"Responder": 1, "Non-responder": 0}).astype(np.int8)
X_multi = X_multi_full[list(tr_top) + list(fl_top)]
print("✅ Data loaded and selected.")


✅ Data loaded and selected.


In [13]:
# Optuna Objective Functions
def tune_logreg(trial, X_train, X_val, y_train, y_val):
    C = trial.suggest_float("C", 1e-3, 100, log=True)
    model = LogisticRegression(solver="lbfgs", max_iter=1000, C=C)
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, preds)


def tune_rf(trial, X_train, X_val, y_train, y_val):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 800, step=200),
        "max_depth": trial.suggest_int("max_depth", 6, 16),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 6),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
    }
    model = RandomForestClassifier(random_state=SEED, n_jobs=-1, **params)
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, preds)


def tune_xgb(trial, X_train, X_val, y_train, y_val):
    params = {
        "objective": "binary:logistic",
        "eval_metric": "auc",
        "tree_method": "hist",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 4, 12),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000, step=200)
    }
    model = xgb.XGBClassifier(**params, n_jobs=-1, random_state=SEED, verbosity=0)
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_val)[:, 1]
    return roc_auc_score(y_val, preds)


def tune_ann(trial, X_train, X_val, y_train, y_val):
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.2, 0.6)
    n1 = trial.suggest_int("n1", 32, 128, step=32)
    n2 = trial.suggest_int("n2", 16, 64, step=16)

    model = models.Sequential([
        layers.Input(shape=(X_train.shape[1],)),
        layers.Dense(n1, activation="relu"),
        layers.Dropout(dropout),
        layers.Dense(n2, activation="relu"),
        layers.Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer=optimizers.Adam(learning_rate=lr),
                  loss="binary_crossentropy", metrics=["accuracy"])
    early = EarlyStopping(patience=5, restore_best_weights=True)
    model.fit(X_train, y_train, validation_data=(X_val, y_val),
              epochs=30, batch_size=32, verbose=0, callbacks=[early])
    preds = model.predict(X_val)
    return roc_auc_score(y_val, preds)


In [14]:
# Build Models (Logistic Regression, Random Forest, XGBoost and ANN)
def build_models(X_train, y_train):
    if not TUNE_MODE:
        return {
            "LogReg": (LogisticRegression(C=0.5, solver="lbfgs", max_iter=1000), None),
            "RandomForest": (RandomForestClassifier(n_estimators=500, max_depth=10, n_jobs=-1, random_state=SEED), None),
            "XGBoost": (xgb.XGBClassifier(learning_rate=0.05, max_depth=8, n_estimators=800,
                                          subsample=0.9, colsample_bytree=0.8, n_jobs=-1, verbosity=0), None),
            "ANN": (None, None)
        }
    else:
        X_t, X_v, y_t, y_v = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=SEED)
        print("🎯 Running Optuna tuning for all models...")
        models_out = {}

        for name, tuner in zip(
            ["LogReg", "RandomForest", "XGBoost", "ANN"],
            [tune_logreg, tune_rf, tune_xgb, tune_ann]
        ):
            study = optuna.create_study(direction="maximize")
            study.optimize(lambda trial: tuner(trial, X_t, X_v, y_t, y_v), n_trials=N_TRIALS, show_progress_bar=True)
            print(f"✅ {name} best AUC={study.best_value:.3f} | params={study.best_params}")

            if name == "LogReg":
                model = LogisticRegression(**study.best_params, solver="lbfgs", max_iter=1000)
            elif name == "RandomForest":
                model = RandomForestClassifier(**study.best_params, random_state=SEED, n_jobs=-1)
            elif name == "XGBoost":
                model = xgb.XGBClassifier(**study.best_params, n_jobs=-1, random_state=SEED, verbosity=0)
            else:
                model = None  # ANN uses params only

            models_out[name] = (model, study.best_params)

        return models_out


In [15]:
# Main Training Loop
results = []
modalities = {
    "Transcriptomic (250)": (X_tr_250, y_trans),
    "Fluxomic (200)": (X_fl_200, y_flux),
    "Multimodal (250+200)": (X_multi, y_multi)
}

for label, (X, y) in modalities.items():
    print(f"\n==============================\n🧬 {label}\n==============================")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    models_dict = build_models(X_train_scaled, y_train)

    for model_name, (model, best_params) in models_dict.items():
        print(f"\n🚀 Training {model_name} ...")

        if model_name == "ANN":
            params = best_params if best_params else {"lr": 0.001, "dropout": 0.5, "n1": 64, "n2": 32}
            ann = models.Sequential([
                layers.Input(shape=(X_train_scaled.shape[1],)),
                layers.Dense(params["n1"], activation="relu"),
                layers.Dropout(params["dropout"]),
                layers.Dense(params["n2"], activation="relu"),
                layers.Dense(1, activation="sigmoid")
            ])
            ann.compile(optimizer=optimizers.Adam(learning_rate=params["lr"]),
                        loss="binary_crossentropy", metrics=["accuracy"])
            early_stop = EarlyStopping(patience=5, restore_best_weights=True)
            ann.fit(X_train_scaled, y_train, validation_data=(X_test_scaled, y_test),
                    epochs=30, batch_size=32, verbose=0, callbacks=[early_stop])
            y_prob = ann.predict(X_test_scaled)
            y_pred = (y_prob > 0.5).astype(int)
            acc = accuracy_score(y_test, y_pred)
            prec = precision_score(y_test, y_pred)
            rec = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_prob)
            mcc = matthews_corrcoef(y_test, y_pred)
            brier = brier_score_loss(y_test, y_prob)
        else:
            model.fit(X_train_scaled, y_train)
            acc, prec, rec, f1, auc, mcc, brier = evaluate_model(model, X_test_scaled, y_test, model_name)

        result = {
            "Modality": label,
            "Model": model_name,
            "Acc": acc, "Prec": prec, "Rec": rec, "F1": f1,
            "AUC": auc, "MCC": mcc, "Brier": brier
        }

        if best_params:
            result["Best_Params"] = str(best_params)

        # SHAP for multimodal XGBoost
        if label == "Multimodal (250+200)" and model_name == "XGBoost":
            print("🧩 Computing SHAP contributions...")
            sample_X = pd.DataFrame(X_test_scaled, columns=X.columns).sample(n=min(100, len(X_test)), random_state=42)
            explainer = shap.TreeExplainer(model)
            shap_vals = explainer.shap_values(sample_X)
            shap_df = pd.DataFrame({
                "Feature": sample_X.columns,
                "MeanAbsSHAP": np.abs(shap_vals).mean(axis=0)
            })
            shap_df["Modality"] = ["Fluxomic" if f in X_flux.columns else "Transcriptomic" for f in shap_df["Feature"]]
            contrib = shap_df.groupby("Modality")["MeanAbsSHAP"].sum()
            perc = 100 * contrib / contrib.sum()
            result["SHAP_Tr_%"] = perc.get("Transcriptomic", 0)
            result["SHAP_Fl_%"] = perc.get("Fluxomic", 0)
            print(f"📊 SHAP → Tr: {perc.get('Transcriptomic',0):.1f}%, Fl: {perc.get('Fluxomic',0):.1f}%")

        results.append(result)
        gc.collect()



🧬 Transcriptomic (250)


[I 2025-10-13 23:06:37,390] A new study created in memory with name: no-name-d44b7014-9284-47c6-b4fc-13d970fcb650


🎯 Running Optuna tuning for all models...


Best trial: 0. Best value: 0.870769:   5%|▌         | 1/20 [00:00<00:18,  1.01it/s]

[I 2025-10-13 23:06:38,382] Trial 0 finished with value: 0.8707689630813163 and parameters: {'C': 0.00438265076200387}. Best is trial 0 with value: 0.8707689630813163.


...

Best trial: 13. Best value: 0.891083:  95%|█████████▌| 19/20 [09:21<00:27, 27.62s/it]

[I 2025-10-14 00:19:46,805] Trial 18 finished with value: 0.8835439418281194 and parameters: {'lr': 0.004337442645770172, 'dropout': 0.3581790439342468, 'n1': 96, 'n2': 32}. Best is trial 13 with value: 0.8910833527246685.
257/257 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


Best trial: 19. Best value: 0.891302: 100%|██████████| 20/20 [09:56<00:00, 29.84s/it]

[I 2025-10-14 00:20:21,624] Trial 19 finished with value: 0.8913016712674601 and parameters: {'lr': 0.0007382544471194025, 'dropout': 0.4376267553395228, 'n1': 128, 'n2': 16}. Best is trial 19 with value: 0.8913016712674601.
✅ ANN best AUC=0.891 | params={'lr': 0.0007382544471194025, 'dropout': 0.4376267553395228, 'n1': 128, 'n2': 16}

🚀 Training LogReg ...


✅ LogReg: acc=0.789, f1=0.673, auc=0.868, mcc=0.519, brier=0.139

🚀 Training RandomForest ...
✅ RandomForest: acc=0.789, f1=0.655, auc=0.873, mcc=0.511, brier=0.146

🚀 Training XGBoost ...
✅ XGBoost: acc=0.838, f1=0.754, auc=0.916, mcc=0.634, brier=0.111
🧩 Computing SHAP contributions...
📊 SHAP → Tr: 82.9%, Fl: 17.1%

🚀 Training ANN ...
321/321 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


In [16]:
results_df = pd.DataFrame(results)
file = results_dir / f"model_results_optuna_extmetrics_{datetime.now().strftime('%Y%m%d_%H%M')}.csv"
results_df.to_csv(file, index=False)
print(f"\n💾 Results saved to {file}")
display(results_df.sort_values(["Modality", "AUC"], ascending=[True, False]))


💾 Results saved to /home/sadegh/python_projects/HRplus-BC-Multimodal/notebooks/results/model_results_optuna_extmetrics_20251014_0022.csv


,Modality,Model,Acc,Prec,Rec,F1,AUC,MCC,Brier,Best_Params,SHAP_Tr_%,SHAP_Fl_%
6,Fluxomic (200),XGBoost,0.663451,0.000000,0.000000,0.000000,0.588718,-0.006994,0.218600,"{'learning_rate': 0.010318199377710972, 'max_d...",NaN,NaN
5,Fluxomic (200),RandomForest,0.663548,0.000000,0.000000,0.000000,0.585916,0.000000,0.219221,"{'n_estimators': 800, 'max_depth': 8, 'min_sam...",NaN,NaN
7,Fluxomic (200),ANN,0.663065,0.419355,0.003727,0.007388,0.565696,0.009609,0.220656,"{'lr': 0.00145440974522045, 'dropout': 0.25391...",NaN,NaN
4,Fluxomic (200),LogReg,0.662680,0.333333,0.002580,0.005121,0.552913,-0.000337,0.221946,{'C': 2.678867629987083},NaN,NaN
10,Multimodal (250+200),XGBoost,0.837896,0.769648,0.739797,0.754427,0.916341,0.633791,0.111492,"{'learning_rate': 0.03694002866407103, 'max_de...",82.936417,17.063589
11,Multimodal (250+200),ANN,0.816074,0.741448,0.696382,0.718209,0.896653,0.582523,0.123538,"{'lr': 0.0007382544471194025, 'dropout': 0.437...",NaN,NaN
9,Multimodal (250+200),RandomForest,0.789187,0.729634,0.593632,0.654644,0.872576,0.510797,0.146350,"{'n_estimators': 800, 'max_depth': 15, 'min_sa...",NaN,NaN
8,Multimodal (250+200),LogReg,0.788894,0.703155,0.645152,0.672906,0.867557,0.518533,0.139468,{'C': 0.00770943859741174},NaN,NaN
2,Transcriptomic (250),XGBoost,0.844043,0.763334,0.758032,0.760674,0.921234,0.645019,0.108139,"{'learning_rate': 0.07264826468857866, 'max_de...",NaN,NaN
3,Transcriptomic (250),ANN,0.828996,0.733428,0.749349,0.741303,0.908638,0.613691,0.115443,"{'lr': 0.0017147475414376616, 'dropout': 0.397...",NaN,NaN


In [ ]:
# # Get models
# def get_models():
#     models_dict = {}

#     # Logistic Regression
#     models_dict["LogReg"] = LogisticRegression(
#         solver="lbfgs", C=0.5, max_iter=1000
#     )

#     # Random Forest
#     models_dict["RandomForest"] = RandomForestClassifier(
#         n_estimators=500, max_depth=10, min_samples_split=2, 
#         min_samples_leaf=1, max_features="sqrt", n_jobs=-1, random_state=42
#     )

#     # XGBoost
#     models_dict["XGBoost"] = xgb.XGBClassifier(
#         objective="binary:logistic", 
#         eval_metric="logloss",
#         learning_rate=0.05,
#         n_estimators=800,
#         max_depth=8,
#         subsample=0.9,
#         colsample_bytree=0.8,
#         tree_method="hist",
#         n_jobs=-1,
#         random_state=42,
#         verbosity=0
#     )

#     # ANN
#     def build_ann(input_dim):
#         model = models.Sequential([
#             layers.Input(shape=(input_dim,)),
#             layers.Dense(64, activation="relu"),
#             layers.BatchNormalization(),
#             layers.Dropout(0.5),
#             layers.Dense(32, activation="relu"),
#             layers.BatchNormalization(),
#             layers.Dense(1, activation="sigmoid")
#         ])
#         model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
#                       loss="binary_crossentropy",
#                       metrics=["accuracy"])
#         return model
#     models_dict["ANN_builder"] = build_ann

#     return models_dict


In [ ]:
# # Train and Test Models for three Modes (Transcriptomics and Fluxomics unimodal vs multimodal)
# results = []
# models_dict = get_models()

# modalities = {
#     "Transcriptomic (250)": (X_tr_250, y_trans),
#     "Fluxomic (200)": (X_fl_200, y_flux),
#     "Multimodal (250+200)": (X_multi, y_multi)
# }

# for label, (X, y) in modalities.items():
#     print(f"\n==============================\n🧬 {label}\n==============================")
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

#     scaler = StandardScaler()
#     X_train_scaled = scaler.fit_transform(X_train)
#     X_test_scaled = scaler.transform(X_test)

#     for model_name, model in models_dict.items():
#         print(f"\n🚀 Training {model_name} ...")

#         if model_name == "ANN_builder":
#             ann = model(X_train_scaled.shape[1])
#             early_stop = EarlyStopping(patience=10, restore_best_weights=True)
#             ann.fit(
#                 X_train_scaled, y_train,
#                 validation_data=(X_test_scaled, y_test),
#                 epochs=50,
#                 batch_size=32,
#                 verbose=0,
#                 callbacks=[early_stop]
#             )
#             y_pred = (ann.predict(X_test_scaled) > 0.5).astype(int)
#             y_prob = ann.predict(X_test_scaled)
#             acc = accuracy_score(y_test, y_pred)
#             prec = precision_score(y_test, y_pred)
#             rec = recall_score(y_test, y_pred)
#             f1 = f1_score(y_test, y_pred)
#             auc = roc_auc_score(y_test, y_prob)
#         else:
#             model.fit(X_train_scaled, y_train)
#             acc, prec, rec, f1, auc = evaluate_model(model, X_test_scaled, y_test, model_name)

#         results.append({
#             "Modality": label,
#             "Model": model_name.replace("_builder", ""),
#             "Acc": acc, "Prec": prec, "Rec": rec, "F1": f1, "AUC": auc
#         })

#         # --- SHAP only for Multimodal XGBoost ---
#         if label == "Multimodal (250+200)" and model_name == "XGBoost":
#             print("🧩 Computing SHAP contributions for multimodal XGBoost...")
#             sample_X = pd.DataFrame(X_test_scaled, columns=X.columns).sample(n=min(100, len(X_test)), random_state=42)
#             explainer = shap.TreeExplainer(model)
#             shap_vals = explainer.shap_values(sample_X)
#             shap_df = pd.DataFrame({
#                 "Feature": sample_X.columns,
#                 "MeanAbsSHAP": np.abs(shap_vals).mean(axis=0)
#             })
#             shap_df["Modality"] = ["Fluxomic" if f in X_flux.columns else "Transcriptomic" for f in shap_df["Feature"]]
#             contrib = shap_df.groupby("Modality")["MeanAbsSHAP"].sum()
#             perc = 100 * contrib / contrib.sum()
#             tr_perc, fl_perc = perc.get("Transcriptomic", 0), perc.get("Fluxomic", 0)

#             results[-1]["SHAP_Tr_%"] = tr_perc
#             results[-1]["SHAP_Fl_%"] = fl_perc
#             print(f"📊 SHAP contribution → Transcriptomic: {tr_perc:.1f}%, Fluxomic: {fl_perc:.1f}%")

#         gc.collect()


In [ ]:
# Save results as csv
# pd.DataFrame(results).to_csv(results_dir / 'comparison_results.csv',index=False)